In [1]:
import numpy as np

## 第一题

In [2]:
class Solution1(object):
    @staticmethod
    def f(x):
        A = np.array([[1.5, -0.5], [-0.5, 0.5]])
        b = np.array([-2, 0])
        return np.dot(np.dot(x, A), x) + np.dot(b, x)
    @staticmethod
    def g(x):
        a = np.dot(np.array([3, -1]), x) - 2
        b = np.dot(np.array([-1, 1]), x)
        return np.array([a, b])
    
    @staticmethod
    def G(x):
        return np.array([[3, -1], [-1, 1]])
    
    def get_alpha(self, x, d, rho, MIN=0, MAX=1):
        a, b = MIN, MAX
        alpha = (a + b) / 2
        while True:
            phi_alpha = self.f(x + alpha * d)
            diff_phi0 = np.dot(self.g(x), d)
            phi0 = self.f(x)
            if phi_alpha <= phi0 + alpha * diff_phi0 * rho:
                if phi_alpha >= phi0 + (1 - rho) * alpha * diff_phi0:
                    return alpha
                else:
                    a = alpha
            else:
                b = alpha
            alpha = (a + b) / 2

    def SteepestDescent(self, x0):
        x = x0
        while True:
            d = -self.g(x)
            if np.linalg.norm(d) < 1e-6:
                break
            alpha = self.get_alpha(x, d, 0.1)
            x = x + alpha * d
        return x, self.f(x)
    
    def Newton(self, x0):
        x = x0
        while True:
            _g = self.g(x)
            if np.linalg.norm(_g) < 1e-6:
                break
            _G = self.G(x)
            d = -np.linalg.solve(_G, _g)
            alpha = self.get_alpha(x, d, 0.1)
            x = x + alpha * d
        return x, self.f(x)

In [3]:
def main1():
    x_star, min_value = Solution1().SteepestDescent(np.array([-2, 4]))
    print(f"Steepest Descent: \nx_star: {np.round(x_star, 4)}, min_value: {min_value:.4f}")
    x_star, min_value = Solution1().Newton(np.array([-2, 4]))
    print(f"Newton: \nx_star: {np.round(x_star, 4)}, min_value: {min_value:.4f}")
main1()

Steepest Descent: 
x_star: [1. 1.], min_value: -1.0000
Newton: 
x_star: [1. 1.], min_value: -1.0000


##  第十题

In [4]:
import sys


class Solution10(object):
    @staticmethod
    def f(x):
        return 100 * np.power(x[1] - x[0] * x[0], 2) + np.power(1 - x[0], 2)
    @staticmethod
    def g(x):
        return np.array([400 * x[0] * (x[0] * x[0] - x[1]) - 2 * (1 - x[0]), 
                         200 * (x[1] - x[0] * x[0])])
    
    @staticmethod
    def G(x):
        return np.array([[1200 * x[0] * x[0] - 400 * x[1] + 2, -400 * x[0]], 
                         [-400 * x[0], 200]])
    
    def get_alpha(self, x, d, rho, MIN=0, MAX=1):
        a, b = MIN, MAX
        alpha = (a + b) / 2
        while np.abs(a - b) > 1e-6:
            phi_alpha = self.f(x + alpha * d)
            diff_phi0 = np.dot(self.g(x), d)
            phi0 = self.f(x)
            if phi_alpha <= phi0 + alpha * diff_phi0 * rho:
                if phi_alpha >= phi0 + (1 - rho) * alpha * diff_phi0:
                    break
                else:
                    a = alpha
            else:
                b = alpha
            alpha = (a + b) / 2
        return alpha

    def SteepestDescent(self, x0):
        x = x0
        while True:
            d = -self.g(x)
            if np.linalg.norm(d) < 1e-6:
                break
            alpha = self.get_alpha(x, d, 0.1)
            x = x + alpha * d
        return x, self.f(x)
    
    def Newton(self, x0):
        x = x0
        while True:
            _g = self.g(x)
            if np.linalg.norm(_g) < 1e-6:
                break
            _G = self.G(x)
            d = -np.linalg.solve(_G, _g)
            alpha = self.get_alpha(x, d, 0.1)
            x = x + alpha * d
        return x, self.f(x)
    
    def ConjugateGradient(self, x0):
        x = x0
        f = self.f(x)
        g = self.g(x)
        d = -g
        while True:
            if np.linalg.norm(g) < 1e-6:
                break
            alpha = self.get_alpha(x, d, 0.1)
            x = x + alpha * d
            f = self.f(x)
            g_new = self.g(x)
            beta = np.dot(g_new, g_new) / np.dot(g, g)
            g = g_new
            d = -g + beta * d
        return x, f
    
    def QuasiNewton(self, x0):
        x = x0.T
        B = np.eye(2)
        g = self.g(x)
        while True:
            if np.linalg.norm(g) < 1e-6:
                break
            d = -np.linalg.solve(B, g)
            alpha = self.get_alpha(x, d, 0.1)
            x_new = x + alpha * d
            g_new = self.g(x_new)
            s = x_new - x
            y = g_new - g
            Bs = B @ s.T
            sy = np.dot(s, y)
            if sy > 1e-10:
                B = B + np.outer(y, y) / sy - np.outer(Bs, Bs) / np.dot(s, Bs)
            x, g = x_new, g_new
        return x, self.f(x)
    

In [5]:
def main10():
    x_star, min_value = Solution10().SteepestDescent(np.array([-1.2, 1]))
    print(f"Steepest Descent: \nx_star: {np.round(x_star, 4)}, min_value: {min_value:.4f}")
    x_star, min_value = Solution10().Newton(np.array([-1.2, 1]))
    print(f"Newton: x_star: \nx_star: {np.round(x_star, 4)}, min_value: {min_value:.4f}")
    x_star, min_value = Solution10().ConjugateGradient(np.array([-1.2, 1]))
    print(f"Conjugate Gradient: x_star: \nx_star: {np.round(x_star, 4)}, min_value: {min_value:.4f}")
    x_star, min_value = Solution10().QuasiNewton(np.array([-1.2, 1]))
    print(f"Quasi-Newton: x_star: \nx_star: {np.round(x_star, 4)}, min_value: {min_value:.4f}")
main10()

Steepest Descent: 
x_star: [1. 1.], min_value: 0.0000
Newton: x_star: 
x_star: [1. 1.], min_value: 0.0000
Conjugate Gradient: x_star: 
x_star: [1. 1.], min_value: 0.0000
Quasi-Newton: x_star: 
x_star: [1. 1.], min_value: 0.0000
